This notebook has some tests and tries to better understand the files generated by snudda, and the organization of info inside them.
when the functions are fully formed they are stored on the getData.py script, to be used as a library

In [1]:
import numpy as np
import h5py
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
#plt.scatter(self.time-skipTime,self.spikeID,color='black',s=1)

# 1) SYNAPSE-RELATED PLOTS (CONN PROB, MATRIX...)
## 1.1) NETWORK-PRUNED-SYNAPSES.HDF5
 Exploring the file to understand the organization of the data inside, so we can collect all the info we need for the plots 

In [5]:
with h5py.File('Snudda/networks/tinySim/network-pruned-synapses.hdf5', 'r') as file:
    network = file.get('network')
    synapses = network.get('synapses')
    neurons = network.get('neurons')
    neuronID = np.array(neurons.get('neuronID'))
    morphology = np.array(neurons.get('morphology'))
    
    syn_table = []
    for i in range(0,len(synapses)):
        syn_table.append(synapses[i])

    df2 = pd.DataFrame(data=syn_table)
    df2.columns = ['sourceID', 'destinID', '3', '4', '5', 'synapseType', '7', 'dentDistance', '9', '10', '11', '12', '13']
    print(df2)    

      sourceID  destinID    3    4    5  synapseType   7  dentDistance    9  \
0            3         0   75  136  199            6  14           240  143   
1            3         0   91  132  211            7  14           260  107   
2            3         0  111  136  185           26  14           192   40   
3            6         0  117  116  178           26  14           366  115   
4            6         0  128  118  167           26  14           289  130   
...        ...       ...  ...  ...  ...          ...  ..           ...  ...   
7029        89        98  142  104  186           26  14           223   92   
7030        89        98  159  135  187           26  14           312   53   
7031        92        98  135   94  176           21  14           239  136   
7032        92        98  135   98  177           21  14           149  120   
7033        92        98  136   98  177           21  14           156  122   

      10   11   12      13  
0      5  960  191  55

## 1.2) FUNCTIONS
Some of the functions that are needed to generate the plots. there are still some missing

In [38]:
import math
            
#Function to find distance => useful for the connection probability plots -> needs to be modified still
def distance(coord1, coord2):
      
    d = math.sqrt((coord2[0] - coord1[0])**2 + (coord2[1] - coord1[1]) **2 + (coord2[2] - coord1[2])**2)
    return(d)

def getSynapses(path):
    
    with h5py.File('/home/ubuntu/Snudda/networks/tinySim/network-pruned-synapses.hdf5', 'r') as file:
        synapses = file.get('network').get('synapses')

        syn_table = []
        for i in range(0,len(synapses)):
            syn_table.append([synapses[i][0], synapses[i][1]]) #Pre_ID | Pos_ID

    return(syn_table)  

## THIS CELL AS WELL. TURN IT INTO A FUNCTION -> that provides the distance between somas
#def DistanceMatrix(positions) :

getNeuronPositions('blabla')
synaptic_table = getSynapses('v')
distance_matrix = []

for i in range (0, len(neuron_positions)):   
    for j in range(0, len(neuron_positions)):
        
        #calculate the distance here
        coord_pre = neuron_positions[i]
        coord_pos = neuron_positions[j]
        #just a boolean, 1 for existing connection, 0 to no connection
        #can be replaced with a function that returns the # of synapses between the given pre pos pair
        if ([i,j] in syn_table):
            synapse = 1
        else :
            synapse = 0        
        distance_matrix.append([i,j,distance(coord_pre, coord_pos), synapse])

df4 = pd.DataFrame(data=distance_matrix)
df4.columns = ['ID_pre', 'ID_pos', 'SomaticDistance', 'Synapse']
print(df4)

In [49]:
#this command selects from the df only the neuron pairs where there is a synapse between them

syn1 = df4[df4["Synapse"] == 1]
print(syn1)

      ID1  ID2  SomaticDistance  Synapse
14      0   14         0.000079        1
30      0   30         0.000092        1
53      0   53         0.000104        1
54      0   54         0.000067        1
61      0   61         0.000116        1
...   ...  ...              ...      ...
9784   98   82         0.000039        1
9787   98   85         0.000016        1
9793   98   91         0.000086        1
9794   98   92         0.000050        1
9799   98   97         0.000077        1

[2005 rows x 4 columns]


# 2) PREPARATION OF THE INPUT INFO

We need to transform the information about the snudda generated input into a new input, compatible with the NEST simulator

## 2.1) INPUT-SPIKES.HDF5 FILE

In [13]:
with h5py.File('/home/ubuntu/BasalGanglia/NEURON-data/net_100/input-spikes.hdf5', 'r') as file:
    
    print("Input hdf5 file contents:")
    print(list(file.keys()))
    
    inputs = file.get('input')
    print("'input' is divided by neuron #")
    print(list(inputs.keys()))
    
    neuron_0 = inputs.get('0')
    print("3 different signals for each neuron")
    print(list(neuron_0.keys()))
    
    CorticalBase_0 = neuron_0.get("CorticalBase")
    CorticalSignal_0 = neuron_0.get("CorticalSignal")
    Thalamic_0 = neuron_0.get("Thalamic")
    
    print("EACH SIGNAL HAS THE FOLLOWING PARAMS")
    print(list(CorticalBase_0.keys()))
    
    print(np.array(CorticalBase_0.get('start')))
    print(np.array(CorticalBase_0.get('nSpikes')))
    print(np.array(CorticalBase_0.get('spikes')))
    


Input hdf5 file contents:
['config', 'input']
'input' is divided by neuron #
['0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '4', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '5', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '6', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '7', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '8', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '9', '90', '91', '92', '93', '94', '95', '96', '97', '98']
3 different signals for each neuron
['CorticalBase', 'CorticalSignal', 'Thalamic']
EACH SIGNAL HAS THE FOLLOWING PARAMS
['channelID', 'channelSpikes', 'conductance', 'correlation', 'end', 'freq', 'generator', 'jitter', 'modFile', 'nSpikes', 'parameterFile', 'parameterID', 'parameterList', 'sectionID', 'sectionX', 'spikes', 'start', 'syn

# 3) RASTER PLOT
example of a raster plot code just to show the general logic behind it (needs the input from a csv which is in my email inbox)
-> necessary steps for the raster still:
- calculate firing rate
- create the beams plot on top of the raster

In [ ]:
import matplotlib.pyplot as plt

import numpy as np

from numpy import genfromtxt
#genfromtxt('my_file.csv', delimiter=',')
 

# Set the random seed for data generation

np.random.seed(2)

 

# Create rows of random data with 50 data points simulating rows of spike trains

data = genfromtxt('02-Data Analysis/data/lgn/plain/neuron_06_stimulus_07.csv', delimiter=',')
z=0
i=0
j=0
for i in range(10):
    for j in range(4500):
        if(data[i,j] == 1):
            x1 = [i,i+0.5]
            x2 = [j,j]
            plt.plot(x2,x1,color = 'black')
        else:
            z= z+1
# # Provide the title for the spike raster plot
plt.title('raster plot Neuron-5-Stimulus-7')
# # Give x axis label for the spike raster plot
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)
fig.savefig('test1png.png', dpi=100)
# # Give y axis label for the spike raster plot
plt.ylabel('trial')
# # Display the spike raster plot
plt.show()
